In [ ]:
Install all required libraries before you get started.

# Install required Azure SDKs
# pip install azure-cognitiveservices-speech 
# pip azure-storage-blob azure-ai-textanalytics 
# pip azure-mgmt-resource 
#pip azure-keyvault azure-ai-openai
#!pip install langchain
#!pip install openai
#!pip install PyPDF2
#!pip install faiss-cpu
#!pip install tiktoken
#!pip install langchain-community

Import all the required libraries before you get started
print("################")
print("Import all required libraries ")
print("################")
import os
from azure.storage.blob import BlobServiceClient
from azure.cognitiveservices.speech import SpeechConfig, SpeechRecognizer, AudioConfig
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from azure.ai.openai import OpenAIClient

print("######################")
print("Add all required environment variables")
print("######################")

AZURE_SPEECH_KEY = os.getenv("AZURE_SPEECH_KEY")
AZURE_SPEECH_REGION = os.getenv("AZURE_SPEECH_REGION")
AZURE_BLOB_CONNECTION_STRING = os.getenv("AZURE_BLOB_CONNECTION_STRING")
AZURE_TEXT_ANALYTICS_KEY = os.getenv("AZURE_TEXT_ANALYTICS_KEY")
AZURE_TEXT_ANALYTICS_ENDPOINT = os.getenv("AZURE_TEXT_ANALYTICS_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

print("##################")
print("# Initialize Azure Components")
print("##################")

def init_clients():
    # Initialize Text Analytics Client
    text_analytics_client = TextAnalyticsClient(
        endpoint=AZURE_TEXT_ANALYTICS_ENDPOINT,
        credential=AzureKeyCredential(AZURE_TEXT_ANALYTICS_KEY),
    )
    # Initialize Blob Storage Client
    blob_service_client = BlobServiceClient.from_connection_string(AZURE_BLOB_CONNECTION_STRING)

    # Initialize OpenAI Client
    openai_client = OpenAIClient(
        endpoint=AZURE_OPENAI_ENDPOINT, credential=AzureKeyCredential(AZURE_OPENAI_API_KEY)    )
    return text_analytics_client, blob_service_client, openai_client

All the required setup is done. Now, we have to implement individual layers.
print("###############################")
print("   Layer 1: Input Layer (Speech-to-Text Conversion)        ")
print("###############################")

def speech_to_text(audio_file_path):
    speech_config = SpeechConfig(subscription=AZURE_SPEECH_KEY, region=AZURE_SPEECH_REGION)
    audio_config = AudioConfig(filename=audio_file_path)

    recognizer = SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    print("Recognizing...")
    result = recognizer.recognize_once()
    
    if result.reason == result.Reason.RecognizedSpeech:
        print(f"Recognized text: {result.text}")
        return result.text
    else:
        print("Speech recognition failed.")
        return None


print("######################################")
print("   Layer 2: Data Ingestion Layer (Upload to Azure Blob Storage)          ")
print("######################################")

def upload_to_blob(container_name, blob_name, content):
    blob_service_client = BlobServiceClient.from_connection_string(AZURE_BLOB_CONNECTION_STRING)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    
    blob_client.upload_blob(content)
    print(f"Uploaded content to Blob: {blob_name}")

print("#########################")
print("   Layer 3: Processing Layer (Text Analysis)    ")
print("#########################")

# def analyze_text(text, text_analytics_client):
    result = text_analytics_client.extract_key_phrases(documents=[text])
    if not result.is_error:
        print(f"Key Phrases: {result[0].key_phrases}")
        return result[0].key_phrases
    else:
        print(f"Error in text analysis: {result[0].error}")
        return []
print("###############################")
print("   Layer 4: AI/ML Layer (Summarization with Open       ")
print("###############################")

def summarize_text(text, openai_client):
    response = openai_client.completions.create(
        engine="text-davinci-003",
        prompt=f"Summarize the following medical text:\n\n{text}",
        max_tokens=150,
    )
    print(f"Summary: {response.choices[0].text.strip()}")
    return response.choices[0].text.strip()

print("####################")
print("   # Main Execution Workflow        ")
print("####################")

def main():
    # Initialize Azure Clients
    text_analytics_client, blob_service_client, openai_client = init_clients()

    # Input Layer: Convert Speech to Text
    audio_file_path = "patient_audio.wav"  # Example audio file
    recognized_text = speech_to_text(audio_file_path)

    if recognized_text:
        # Data Ingestion Layer: Upload to Blob Storage
        upload_to_blob(container_name="medical-records",blob_name="raw_text.txt", content=recognized_text)
        # Processing Layer: Analyze Text
        key_phrases = analyze_text(recognized_text, text_analytics_client)

        # AI/ML Layer: Summarization
        summary = summarize_text(recognized_text, openai_client)
        # Final Output
        print("### Final Output ###")
        print(f"Original Text: {recognized_text}")
        print(f"Key Phrases: {key_phrases}")
        print(f"Summary: {summary}")

if __name__ == "__main__":
    main()
